In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq
 
from bokeh.palettes import brewer, Spectral6
from bokeh.io import show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Panel, Tabs, GeoJSONDataSource, LinearColorMapper
from bokeh.models import ColorBar
from bokeh.layouts import column, gridplot, row
from bokeh.transform import factor_cmap
from bokeh.models import NumeralTickFormatter, Title, Label, Paragraph, Div, CustomJSHover, BoxAnnotation

This notebook plots Canadian Aluminum US imports (which were subject to renewed 232 actions recently), it can also be used as a template for product specific plots (like in this case). Some notes:

- This is a good place to see sequencing: https://www.cbp.gov/trade/programs-administration/trade-remedies#

- This is the initial time line Canada does not happen till later: https://content.govdelivery.com/accounts/USDHSCBP/bulletins/1e43e86

- Then Canada goes no line here June 1: https://content.govdelivery.com/accounts/USDHSCBP/bulletins/1f41b4b

- Canada tariff is taken off: https://content.govdelivery.com/accounts/USDHSCBP/bulletins/24602aa

- Tarfiff goes back on https://content.govdelivery.com/accounts/USDHSCBP/bulletins/29a1ddd

In [2]:
trade_type = "imports"

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

In [3]:
crl = ["darkblue","slategray","crimson"]

background = "#ffffff"

In [4]:
last_month = 12

---
#### Grabe the trade data using the Census's API

In [5]:
def census_trade(url, trade_type, country, product_level):
    
    r = requests.get(url) 
    
    print(r)
    
    df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    df.columns = r.json()[0]

    df.time = pd.to_datetime(df.time, format="%Y-%m")
    # This is so I can call this correctly...
    
    if trade_type == "imports":
        
        trade_label = country + "_" + trade_type
        
        df[trade_label] = df["CON_VAL_MO"].astype(float)
        
        df[product_level] = df["NAICS"].astype(str)
        
        df.drop(["CON_VAL_MO", "NAICS", "COMM_LVL"], axis = 1, inplace = True)
        
    if trade_type == "exports":
    
        trade_label = country + "_" + trade_type
        
        df[trade_label] = df["ALL_VAL_MO"].astype(float)

        df[product_level] = df["NAICS"].astype(str)
        
        df.drop(["ALL_VAL_MO", "NAICS", "COMM_LVL"], axis = 1, inplace = True)
        
    if trade_type == "country_exports":
    
        trade_label = "total_exports"
        
        df[trade_label] = df["ALL_VAL_MO"].astype(float)
        
        df = df[df.SUMMARY_LVL == "DET"]
        
        df.drop(["ALL_VAL_MO", "CTY_CODE", "SUMMARY_LVL"], axis = 1, inplace = True)
        
        df = df[df.CTY_NAME != "TOTAL FOR ALL COUNTRIES"]
        
    if trade_type == "country_imports":
    
        trade_label = "total_imports"
        
        df[trade_label] = df["CON_VAL_MO"].astype(float)
        
        #df = df[df.SUMMARY_LVL == "DET"]
        
        df.drop(["CON_VAL_MO", "CTY_CODE"], axis = 1, inplace = True)    
    
    return df

In [6]:
def growth_trade(foo):
    # what this function does is take a dataframe and create a relative 
    
    foo["growth"] = 100*((foo.total_imports/foo.total_imports.shift(12)) - 1)
    
    return foo

In [7]:
def make_trade_time(df):
        
    foo = df.drop(labels = ["naics"], axis = 1).reset_index().pivot(index = "time", columns = "NAICS_LDESC")
    
    foo.drop(labels = ["index"], axis = 1, inplace = True)
    
    return foo

---
### This is the Product Specific plot...

In [13]:
#end_use = "naics?get=NAICS,CON_VAL_MO,CTY_CODE,CTY_NAME,SUMMARY_LVL,NAICS_LDESC"

#url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
#url = url + my_key + "&time==from+2013-01" + "&COMM_LVL=NA4" + "&NAICS=3313"

end_use = "hs?get=CON_VAL_MO,CTY_CODE,CTY_NAME,I_COMMODITY_LDESC"

url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
url = url + my_key + "&time==from+2013-01" + "&COMM_LVL=HS10" + "&I_COMMODITY=76*"


url = url

df = census_trade(url,"country_imports","foo", "foo")

df.time.max()

<Response [500]>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
df.head()

In [ ]:
df = df[["CTY_NAME","time","total_imports"]]

In [ ]:
df = df.groupby(["time", "CTY_NAME"])

df = df.agg({"total_imports": "sum"})



In [ ]:
df.reset_index(inplace = True)

In [ ]:
df.head()

In [ ]:
clist = ["TOTAL FOR ALL COUNTRIES", "CANADA"]

df = df[df.CTY_NAME.isin(clist)]

In [ ]:
df.head()

In [ ]:
grp = df.groupby(["CTY_NAME"])

country_df = grp.apply(growth_trade)

In [ ]:
def make_trade_time_country(df):
        
    foo = df.reset_index().pivot(index = "time", columns = "CTY_NAME")
    
    foo.drop(labels = ["index"], axis = 1, inplace = True)
    
    return foo

In [ ]:
def make_time_by_country(df, level):

    height = 533
    width = 600

    foobar = make_trade_time_country(df)
    
    if level == "total_imports":
        title = "$ US Imports from CANADA (HS 76, ALUMINUM AND ARTICLES THEREOF)"
        
        foobar.drop([ "TOTAL FOR ALL COUNTRIES"], axis = 1, level = 1, inplace = True)
    
    if level == "growth":
        title = "CANADA and TOTAL, % Year over Year Change in US Imports (HS 76, ALUMINUM AND ARTICLES THEREOF)"
    
    p = figure(plot_height=height, plot_width = width, x_axis_type="datetime",toolbar_location = 'below',
           tools = "box_zoom, reset, pan, xwheel_zoom", title = title, x_range = (dt.datetime(2015,1,1),dt.datetime(2020,last_month,1)) ) 

    numlines=len(foobar[level].columns)

    multi_line_source = ColumnDataSource({
        'xs': [foobar.index.values]*numlines,
        'ys': [foobar[level, name].values for name in foobar[level]],
        'label': [name for name in foobar["total_imports"].columns],
        'color': ["crimson", "darkblue"]})

    p.multi_line(xs= "xs",
                ys= "ys",
                line_width=2.5, line_alpha=0.45, line_color = "color",
                 hover_line_alpha=0.75, hover_line_width = 5,
                hover_line_color= "color", source = multi_line_source)
    
    y_custom = CustomJSHover(code=""" return '' + special_vars.data_y
            """)
            
    TIMETOOLTIPS = """
            <div style="background-color:#F5F5F5; opacity: 0.95; border: 5px 5px 5px 5px;">
            <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold"> @label
             </span>
             </div>
             <div style = "text-align:left;">"""
    
    if level == "total_imports":
        TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%b %Y}:  $data_y{$0.0a}</span>   
            </div>
            </div>
            """
    if level == "growth":
        TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%b %Y}:  $data_y{0.0}%</span>   
            </div>
            </div>
            """

    p.add_tools(HoverTool(tooltips = TIMETOOLTIPS,  line_policy='nearest', formatters={'$data_x': 'datetime'}))
    p.title.text_font_size = '13pt'
    p.background_fill_color = background 
    p.background_fill_alpha = 0.75
    p.border_fill_color = background 
    
    
    tariffs232_box = BoxAnnotation(left=dt.datetime(2018,5,31), right=dt.datetime(2019,5,20), fill_color='slategray', fill_alpha=0.1)
    p.add_layout(tariffs232_box)
    
    new_tariffs232_box = BoxAnnotation(left=dt.datetime(2020,8,16), right=dt.datetime(2020,12,31), fill_color='slategray', fill_alpha=0.1)
    p.add_layout(new_tariffs232_box)
    
    #p.yaxis.axis_label = 
    p.yaxis.axis_label_text_font_style = 'bold'
    p.yaxis.axis_label_text_font_size = "13px"

    p.yaxis.minor_tick_line_color = None
    
    if level == "total_imports":
        p.yaxis.formatter = NumeralTickFormatter(format="($0. a)")
        
    if level == "growth":
        p.y_range.end = 75
    
    p.outline_line_color = None
    p.sizing_mode= "scale_both"
    p.max_height = height
    p.max_width = width
    
    p.toolbar.active_drag = None
    p.toolbar.autohide = True
    p.min_border_left = 0
    p.min_border_bottom = 0

    return p

In [ ]:
clevel = make_time_by_country(country_df, "total_imports")
    
cchange = make_time_by_country(country_df, "growth")

tab3 = Panel(child= clevel, title="Imports in $")

tab4 = Panel(child= cchange, title="Imports as % YoY")

output_file('.\\docs\\' + "us_imports_alu.html")

div0 = Div(text = """Shaded grey area indicates when the Section 232 tariffs, under Proclamation 9704, on Canada went into effect
and were removed. Hover over the line with your curser to view month by month values.""", max_width=555, background = background )
div0.sizing_mode= "scale_both"
        
outfig = column(Tabs(tabs=[tab3,  tab4], tabs_location = "above"), div0, sizing_mode="scale_both")

show(outfig)